# Using Stable Baselines 3 Evaluation Callbacks During Training

In [ ]:
from stable_baselines3.common.env_checker import check_env

from yawning_titan.envs.generic.core.blue_interface import BlueInterface
from yawning_titan.envs.generic.core.red_interface import RedInterface
from yawning_titan.envs.generic.generic_env import GenericNetworkEnv
from yawning_titan.envs.generic.helpers import network_creator
from yawning_titan.envs.generic.core.action_loops import ActionLoop
from yawning_titan.envs.generic.core.network_interface import NetworkInterface
from yawning_titan.config.network_config.network_config import NetworkConfig
from yawning_titan.config.game_modes import default_game_mode_path
from yawning_titan.config.game_config.game_mode_config import GameModeConfig

## Built the `NetworkConfig`.

In [ ]:
matrix, node_positions = network_creator.create_mesh(size=20)
network_config = NetworkConfig.create_from_args(matrix=matrix, positions=node_positions)

## Build the `GameModeConfig`.

In [ ]:
game_mode_config = GameModeConfig.create_from_yaml(default_game_mode_path())

## Build the `NetworkInterface`.

In [ ]:
network_interface = NetworkInterface(game_mode=game_mode_config, network=network_config)

In [ ]:
red = RedInterface(network_interface)
blue = BlueInterface(network_interface)

## Create the environment

In [ ]:
env = GenericNetworkEnv(red, blue, network_interface)

In [ ]:
check_env(env, warn=True)
_ = env.reset()

## Initialise the Evalulation Callback

In [ ]:
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor

In [ ]:
eval_callback = EvalCallback(Monitor(env), eval_freq=1000, deterministic=False, render=False)

## Create an agent

In [ ]:
from stable_baselines3 import A2C, DQN, PPO
from stable_baselines3.ppo import MlpPolicy as PPOMlp

In [ ]:
agent = PPO(PPOMlp, env, verbose=1)

## Train the agent and monitor the progress using the Evaluation Callback

In [ ]:
agent.learn(total_timesteps=2000, n_eval_episodes=1, callback=eval_callback)